In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.chdir("../input")
os.listdir()

## Data Read

In [ ]:
df=pd.read_csv("../input/eeg-clean/eeg_clean.csv")

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
print(df["eye"].value_counts())

In [ ]:
df.eye=[1 if each =="Open" else 0 for each in df.eye]

In [ ]:
df.info()

In [ ]:
y = df["eye"].values
X = df.drop(['eye'], axis=1).values

In [ ]:
# Data Standardization 
from sklearn.preprocessing import StandardScaler
Scaler=StandardScaler()
X=Scaler.fit_transform(X)

X[0:3]

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense,Dropout



# Train-Test 
from sklearn.model_selection import train_test_split
# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=0)

In [ ]:
def create_model(optimizer="adam"):
    # create model
    model = Sequential()
    model.add(Dense(32, input_dim=14, activation='relu')) 
    model.add(Dropout(0.1)) # Added droupout to prevent overfitting
    model.add(Dense(20, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='sigmoid'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid')) # Since it has 2 outputs, 'sigmoid' as activation in the output layer
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=["accuracy"])
    return model
model = create_model() 

In [ ]:
train=model.fit(X_train, y_train, epochs=100, batch_size=50, verbose=0,validation_data=(X_test,y_test))

In [ ]:
# plot loss during training
import matplotlib.pyplot as plt
plt.plot(train.history['loss'], label='train')
plt.plot(train.history['val_loss'], label='test')
plt.title('Model Loss')
plt.xlabel('epochs')
plt.ylabel('loss values')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Untunned Scores of the Model
import sklearn.metrics as metrics
y_pred=model.predict_classes(X_test)

In [ ]:
# %%Accuracy

print("Accuracy:",metrics.accuracy_score(y_test,y_pred))

# %%f1 score

print("f1_weighted:",metrics.f1_score(y_test, y_pred,average='weighted'))

In [ ]:
# Grid Search Cross Validation
# GridSearch Cross Validation Parameters
param_grid = {
   
    'epochs': [100,150],
    'batch_size':[32,50],
    'optimizer':['Adam','SGD']
    
}

# create model

# Creating Model Object with KerasClassifier
model_cv = KerasClassifier(build_fn=create_model, verbose=1)


grid = GridSearchCV(estimator=model_cv,  
                    n_jobs=-1, 
                    verbose=1,
                    cv=5,
                    param_grid=param_grid)

grid_cv_model = grid.fit(X_train, y_train,) # Fitting the GridSearch Object on the Train Set


means = grid_cv_model.cv_results_['mean_test_score'] # Mean of test scores
stds = grid_cv_model.cv_results_['std_test_score'] # standard deviations of test scores
params = grid_cv_model.cv_results_['params'] # parameters used
# to print all scores, standard deviations and parameters used
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Printing the Best Parameters as a Result of Grid Search Cross Validation on the Screen
print("Best: %f using %s" % (grid_cv_model.best_score_, grid_cv_model.best_params_))

In [ ]:
# %% Model Tuning- Building a Tuned Model with Best Parameters
# Creating Tuned Model Object with KerasClassifier
cv_model = grid_cv_model.best_estimator_

In [ ]:
#%% K-FOLD
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# K-fold accuracy scores

kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(cv_model, X_test, y_test, cv=kfold,scoring= 'accuracy')

In [ ]:
print('K-fold Cross Validation Accuracy Results: ', results)
print('K-fold Cross Validation Accuracy Results Mean: ', results.mean())

In [ ]:
# K-fold f1 scores
from sklearn.model_selection import KFold


kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(cv_model, X_test,y_test, cv=kfold,scoring="f1_weighted")

In [ ]:
print('K-fold Cross Validation f1_weighted Results: ', results)
print('K-fold Cross Validation f1_weighted Results Mean: ', results.mean())

In [ ]:
# Tuned Model Prediction

y_pred = cv_model.predict(X_test)

In [ ]:
# %% f1 score
import sklearn.metrics as metrics
print("f1_weighted:",metrics.f1_score(y_test, y_pred,average='weighted'))


# %% Accuracy

print("accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
#%% Confusion Matrix and Classification Report
from sklearn.metrics import confusion_matrix, classification_report 

# Classification Report
model_report = classification_report(y_test, y_pred)
print(model_report)

In [ ]:
# Confusion Matrix
# multilabel-indicator is not supported so np.argmax should be used!
model_conf = confusion_matrix(y_test,y_pred)
print(model_conf)

In [ ]:
#%% ROC-AUC Curve
import matplotlib.pyplot as plt



probs=cv_model.predict_proba(X_test)
fpr,tpr,threshold=metrics.roc_curve(y_test,y_pred)
roc_auc=metrics.auc(fpr,tpr)




plt.title("ROC")
plt.plot(fpr,tpr,label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy',  linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()